In [64]:
series_ids = {
    
'CPI':'CUUR0000SA0', 
'Unemployment':'LNS14000000', 
'NF Employment':'CES0000000001', 
'ULC':'PRS85006112', 
'Hourly earnings':'CES0500000003', 
'PPI':'WPUFD4', 
'Shelter':'CUSR0000SAH1', 
'Energy':'CUUR0000SA0E', 
'Food':'CUUR0000SAF1'

}
#CPI, Unemployment, Total nonfarm employment, unit labour costs, hourly earnings, PPI final demand, Shelter Inflation, Energy Inflation, Food Inflation

In [100]:
api_key = '178d3446c4fe4ee388252bdeb9bb243e'  

def create_df(id):
    series_id = id  
    start_year = '2013'
    end_year = '2023'

    url = f'https://api.bls.gov/publicAPI/v2/timeseries/data/'

    data = {
        'seriesid': [series_id],
        'startyear': start_year,
        'endyear': end_year,
        'registrationKey': api_key
    }

    response = requests.post(url, json=data)
    data = response.json()

    # Convert to DataFrame
    df = pd.DataFrame(data['Results']['series'][0]['data'])
    df['value'] = df['value'].astype(float)
    df['yoy'] = 100*((df['value']/df['value'].shift(-12))-1)
    df = df[['year', 'periodName', 'value', 'yoy']]

    return(df)
    

In [102]:

combined_df = pd.DataFrame()


for series_name, series_id in series_ids.items():

    df = create_df(series_id)
    
    df = df.rename(columns={'value': series_name, 'yoy': f'{series_name}_yoy'})

    if combined_df.empty:
        combined_df = df
    else:
        combined_df = pd.merge(combined_df, df, on=['year', 'periodName'], how='outer')

print(combined_df.head())

   year periodName      CPI   CPI_yoy  Unemployment  Unemployment_yoy  \
0  2023   December  306.746  3.352123           3.7          5.714286   
1  2023   November  307.051  3.137271           3.7          2.777778   
2  2023    October  307.671  3.241145           3.8          5.555556   
3  2023  September  307.789  3.699698           3.8          8.571429   
4  2023     August  307.026  3.665112           3.8          5.555556   

   NF Employment  NF Employment_yoy  ULC  ULC_yoy  Hourly earnings  \
0       157232.0           1.745236  NaN      NaN            34.27   
1       157016.0           1.762845  NaN      NaN            34.12   
2       156843.0           1.842136  NaN      NaN            34.00   
3       156738.0           1.988522  NaN      NaN            33.91   
4       156476.0           2.050453  NaN      NaN            33.82   

   Hourly earnings_yoy      PPI   PPI_yoy  Shelter  Shelter_yoy   Energy  \
0             4.100851  141.154  0.982973  390.037     6.174409 

In [103]:
combined_df.head()

,year,periodName,CPI,CPI_yoy,Unemployment,Unemployment_yoy,NF Employment,NF Employment_yoy,ULC,ULC_yoy,Hourly earnings,Hourly earnings_yoy,PPI,PPI_yoy,Shelter,Shelter_yoy,Energy,Energy_yoy,Food,Food_yoy
0,2023,December,306.746,3.352123,3.7,5.714286,157232.0,1.745236,NaN,NaN,34.27,4.100851,141.154,0.982973,390.037,6.174409,269.375,-2.023009,325.409,2.704844
1,2023,November,307.051,3.137271,3.7,2.777778,157016.0,1.762845,NaN,NaN,34.12,4.024390,141.877,0.775651,388.266,6.526010,277.029,-5.435684,325.172,2.949119
2,2023,October,307.671,3.241145,3.8,5.555556,156843.0,1.842136,NaN,NaN,34.00,4.102878,142.448,1.213585,386.536,6.730727,286.754,-4.529580,325.731,3.300742
3,2023,September,307.789,3.699698,3.8,8.571429,156738.0,1.988522,NaN,NaN,33.91,4.242238,142.898,2.021904,385.259,7.145857,296.004,-0.450322,324.704,3.692255
4,2023,August,307.026,3.665112,3.8,5.555556,156476.0,2.050453,NaN,NaN,33.82,4.286155,142.641,1.911907,382.782,7.248886,294.328,-3.616573,324.100,4.254121


In [107]:
combined_df.drop(columns='Unemployment_yoy', inplace=True)
combined_df.head()

,year,periodName,CPI,CPI_yoy,Unemployment,NF Employment,NF Employment_yoy,ULC,ULC_yoy,Hourly earnings,Hourly earnings_yoy,PPI,PPI_yoy,Shelter,Shelter_yoy,Energy,Energy_yoy,Food,Food_yoy
0,2023,December,306.746,3.352123,3.7,157232.0,1.745236,NaN,NaN,34.27,4.100851,141.154,0.982973,390.037,6.174409,269.375,-2.023009,325.409,2.704844
1,2023,November,307.051,3.137271,3.7,157016.0,1.762845,NaN,NaN,34.12,4.024390,141.877,0.775651,388.266,6.526010,277.029,-5.435684,325.172,2.949119
2,2023,October,307.671,3.241145,3.8,156843.0,1.842136,NaN,NaN,34.00,4.102878,142.448,1.213585,386.536,6.730727,286.754,-4.529580,325.731,3.300742
3,2023,September,307.789,3.699698,3.8,156738.0,1.988522,NaN,NaN,33.91,4.242238,142.898,2.021904,385.259,7.145857,296.004,-0.450322,324.704,3.692255
4,2023,August,307.026,3.665112,3.8,156476.0,2.050453,NaN,NaN,33.82,4.286155,142.641,1.911907,382.782,7.248886,294.328,-3.616573,324.100,4.254121


In [109]:
combined_df.tail()

,year,periodName,CPI,CPI_yoy,Unemployment,NF Employment,NF Employment_yoy,ULC,ULC_yoy,Hourly earnings,Hourly earnings_yoy,PPI,PPI_yoy,Shelter,Shelter_yoy,Energy,Energy_yoy,Food,Food_yoy
171,2014,1st Quarter,NaN,NaN,NaN,NaN,NaN,12.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2013,4th Quarter,NaN,NaN,NaN,NaN,NaN,-0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,2013,3rd Quarter,NaN,NaN,NaN,NaN,NaN,-3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,2013,2nd Quarter,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,2013,1st Quarter,NaN,NaN,NaN,NaN,NaN,-8.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 175
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 176 non-null    object 
 1   periodName           176 non-null    object 
 2   CPI                  132 non-null    float64
 3   CPI_yoy              120 non-null    float64
 4   Unemployment         132 non-null    float64
 5   NF Employment        132 non-null    float64
 6   NF Employment_yoy    120 non-null    float64
 7   ULC                  44 non-null     float64
 8   ULC_yoy              32 non-null     float64
 9   Hourly earnings      132 non-null    float64
 10  Hourly earnings_yoy  120 non-null    float64
 11  PPI                  132 non-null    float64
 12  PPI_yoy              120 non-null    float64
 13  Shelter              132 non-null    float64
 14  Shelter_yoy          120 non-null    float64
 15  Energy               132 non-null    flo

In [111]:
combined_df.to_csv('first_data.csv')